In [ ]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

In [ ]:
!pip install tensorflow

In [ ]:
data_in = pd.read_csv("../data/mushrooms.csv")

In [ ]:
data_in.head()

In [ ]:
# probably have to do a specific type of sampling to make sure we don't end up accidentally overrepresenting one group. 
# Not sure about the rest of the stuff. 

# This notebook will be mostly used as an example for how to do some Tensorflow models - I don't care about the classification itself
data_in.groupby("Class").count()

### Test / train datasets creation

In [ ]:
X = data_in.copy()
y = X.pop("Class")


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [ ]:
num_epochs = 50
df = pd.concat([X_train, y_train], axis=1)

In [ ]:
df

In [ ]:
ALL_COLUMNS = df.columns
FEATURES = ALL_COLUMNS[1:len(ALL_COLUMNS) - 1]
LABEL = "Class"

In [ ]:
# 

def make_train_input_fn(df, num_epochs):
  return tf.compat.v1.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

  
def make_eval_input_fn(df):
  return tf.compat.v1.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

  
def make_prediction_input_fn(df):
  return tf.compat.v1.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

In [ ]:
## Create features

In [ ]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

In [ ]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

OUTDIR = 'output'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_train_input_fn(df_train, num_epochs = 10))